In [39]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from konlpy.tag import Okt
from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
base_url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm'

cat_sid = [101, 102, 103, 104, 105] #카테고리 대분류
duration = 1 #수집 기간

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

In [3]:
def is_final_page(url, page):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    paging = soup.select('.paging')
    page_list = paging[0].text.split("\n")
    
    if '다음' in page_list:
        return False
    else:
        page_num = []
        for item in page_list:
            try:
                page_num.append(int(item))
            except:
                pass
        if max(page_num) == page:
            return True
    
    return False

In [4]:
def generate_date_list(start_date, end_date):
    date_list = []
    current_date = start_date

    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        date_list.append(date_str)
        current_date += timedelta(days=1)

    return date_list

# 시작 날짜와 끝 날짜 설정
today = datetime.now()
start_date = today - timedelta(days=duration)  # 시작 날짜
end_date = today - timedelta(days=1)    # 끝 날짜

# 날짜 리스트 생성
date_list = generate_date_list(start_date, end_date)

In [5]:
url_lists = []
for cat in cat_sid:
    for date in date_list:
        url = base_url + '&sid1=' + str(cat) + '&date=' + date
        
    
        url_dic = {'sid':cat, 'date':date, 'url':url}
        url_lists.append(url_dic)

url_lists_df = pd.DataFrame(url_lists)

In [6]:
def get_news_url_lists(sid, date, page, base_url):
    url = base_url + '&sid=' + str(sid) + '&date=' + date + '&page=' + str(page)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = soup.select('.list_body')
    news_links = news_list[0].find_all('a')
    news_urls = []

    for link in news_links:
        news_dic = {'sid':sid, 'date':date, 'news_url':link.get("href")}
        news_urls.append(news_dic)

    news_urls_df = pd.DataFrame(news_urls)
    news_urls_df.drop_duplicates(inplace=True)
    news_urls_df.reset_index(inplace=True)
    
    return news_urls_df, url

In [33]:
news_urls_df = pd.DataFrame()

for index, row in tqdm(url_lists_df.iterrows(), desc='process'):
    is_final = False
    i = 1
    
    while(not is_final):
        news_urls_temp_df, merged_url = get_news_url_lists(row.sid, row.date, i, base_url)
        news_urls_df = pd.concat([news_urls_df, news_urls_temp_df], ignore_index=True)

        is_final = is_final_page(merged_url, i)
        i+= 1

process: 5it [27:57, 335.57s/it]


In [36]:
news_urls_df.to_csv('result_news_urls.csv')

In [7]:
news_urls_df = pd.read_csv('result_news_urls.csv')

In [8]:
article_list = []
i = 0

for link in tqdm(news_urls_df['news_url'], desc = 'Process'):
    try:
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        date_html = soup.select('.media_end_head_info_datestamp_time')
        date = date_html[0].get('data-date-time')
        title = soup.select('.media_end_head_title')[0].text.replace('\n', '')
        document = soup.select('.newsct_article')[0].text.replace('\n', '')
        press = soup.select('.u_cra_v1')[1].text
        category = soup.select('.media_end_categorize_item')[0].text
        

        article_dic = {'date':date, 'press':press, 'category':category, 'title':title, 'document':document, 'link':link}
        article_list.append(article_dic)
    except:
        print(str(i) + 'error')
        pass
    
    i+= 1

article_list_df = pd.DataFrame(article_list)
article_list_df.reset_index(inplace=True)

Process:   0%|          | 0/67595 [00:00<?, ?it/s]

0error
1error
2error
3error


Process:   0%|          | 24/67595 [00:05<4:01:20,  4.67it/s]

24error
25error
26error
27error


Process:   0%|          | 48/67595 [00:10<4:23:47,  4.27it/s]

...

67564error
67565error
67566error
67567error


Process: 100%|█████████▉| 67588/67595 [4:17:35<00:01,  4.06it/s]

67588error
67589error
67590error
67591error


Process: 100%|██████████| 67595/67595 [4:17:36<00:00,  4.37it/s]

67593error
67594error


In [10]:
article_list_df.to_csv('result_naver_news.csv')

In [12]:
set(article_list_df['category'])

{'IT', '경제', '사회', '생활', '세계', '오피니언', '정치'}

In [37]:
article_list_df.head()

,index,date,press,category,title,document,link
0,0,2023-10-09 23:59:19,KBS 뉴스,사회,북한 함경북도 길주에서 규모 2.2 지진,오늘일밤시분에북한함경북도길주에서규모의지진이발생했습니다지진의진앙은북한함경북도길주북북서...,https://n.news.naver.com/mnews/article/056/001...
1,1,2023-10-09 23:58:07,AP연합뉴스,세계,Lebanon Israel Palestinians,,https://n.news.naver.com/mnews/article/077/000...
2,2,2023-10-09 23:58:05,AP연합뉴스,세계,NerdWallet-Kimberly-Palmer-Planning-for-Inheri...,,https://n.news.naver.com/mnews/article/077/000...
3,3,2023-10-09 23:58:01,동아일보,오피니언,"[사설]北 장사정포·특작부대, 核에 가려있던 기습 위협 다시 볼 때",지난주말팔레스타인무장단체하마스의대규모로켓공격은이스라엘의첨단요격방어시스템아이언돔을무력...,https://n.news.naver.com/mnews/article/020/000...
4,4,2023-10-09 23:57:11,연합뉴스,사회,"기상청 ""북한 함경북도 길주 북북서쪽서 2.2 지진…자연지진""",지진발생위치기상청홈페이지서울연합뉴스일오후시분초북한함경북도길주북북서쪽지역에서규모의지진...,https://n.news.naver.com/mnews/article/001/001...


In [46]:
article_list_df['document'] = article_list_df['document'].str.replace(pat=r'[^ㄱ-ㅣ가-힣]+', repl= r'', regex=True)
model_data = article_list_df[article_list_df['document'].astype(bool)]

In [66]:
model_data.to_csv('learning_data.csv')

In [2]:
model_data = pd.read_csv('learning_data.csv')

In [38]:
model_data['title'] = model_data['title'].str.replace(pat=r'[^ㄱ-ㅣ가-힣]+', repl= r'', regex=True)

In [8]:
okt = Okt()
def token_generate(doc):
    return okt.nouns(doc)

In [12]:
new_doc_x = model_data['title']
new_doc_y = model_data['category_label']

In [13]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(new_doc_x, new_doc_y, test_size=0.3, random_state=0)

In [14]:
svc_pipe = make_pipeline(TfidfVectorizer(tokenizer=token_generate),SVC(kernel='rbf', C=10, gamma=0.1))

In [15]:
svc_pipe.fit(x_train, y_train)

/Users/lanoon/anaconda3/envs/AI_NLP/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function token_generate at 0x14537bee0>)),
                ('svc', SVC(C=10, gamma=0.1))])

In [40]:
logistic_pipe = make_pipeline(TfidfVectorizer(tokenizer=token_generate),LogisticRegression())

In [41]:
logistic_pipe.fit(x_train,y_train)

/Users/lanoon/anaconda3/envs/AI_NLP/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/lanoon/anaconda3/envs/AI_NLP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function token_generate at 0x14537bee0>)),
                ('logisticregression', LogisticRegression())])

In [17]:
print("SVC Accuracy on Training set: {:.3f}".format(svc_pipe.score(x_train, y_train)))
print("SVC Accuracy on Test set: {:.3f}".format(svc_pipe.score(x_test, y_test)))

Accuracy on Training set: 0.981
Accuracy on Test set: 0.974


In [42]:
print("Logistic Accuracy on Training set: {:.3f}".format(logistic_pipe.score(x_train, y_train)))
print("Logistic Accuracy on Test set: {:.3f}".format(logistic_pipe.score(x_test, y_test)))

Logistic Accuracy on Training set: 0.945
Logistic Accuracy on Test set: 0.914


In [29]:
def category_answer(text):
    input_text = []
    input_text.append(text)
    if int(svc_pipe.predict(input_text)) == 1:
        print('IT')
    elif int(svc_pipe.predict(input_text)) == 2:
        print('경제')
    elif int(svc_pipe.predict(input_text)) == 3:
        print('사회')
    elif int(svc_pipe.predict(input_text)) == 4:
        print('생활')
    elif int(svc_pipe.predict(input_text)) == 5:
        print('오피니언')
    elif int(svc_pipe.predict(input_text)) == 6:
        print('세계')
    else:
        print('정치')

In [37]:
category_answer('한국소프트웨어산업협회(KOSA), 국내 SW 스타트업 중동 진출 돕는다')

IT


In [43]:
logistic_pipe.predict(['태풍 독수리 북상중 한반도 장마 영향 가능성'])

array([3])